In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install -q transformers
%pip install -q loguru
%pip install -q openai langchain langchain[docarray] tiktoken

In [2]:
!wget -O test.mp3 https://dl.dropbox.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq&dl=0

--2023-11-10 20:58:23--  https://dl.dropbox.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.81.15, 2620:100:6031:15::a27d:510f
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.81.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq [following]
--2023-11-10 20:58:24--  https://dl.dropboxusercontent.com/scl/fi/ri0koqljazuoj7irxcx14/Jeff-Bezos-on-the-Importance-of-Stress-Management.mp3?rlkey=kbvm6wvf3n41c6j2u558kx4jq
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6025:15::a27d:450f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.81.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 720916 (7

In [4]:
import os

import torch
from rich.pretty import pprint
from transformers import pipeline

from loguru import logger

import openai
import sys

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pprint(device)

'cuda'

In [6]:
pipe = pipeline(
    "automatic-speech-recognition",
    "openai/whisper-large-v3",
    torch_dtype=torch.float16,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
audio_file_path = os.path.join("test.mp3")

if not os.path.exists(audio_file_path):
    logger.error(f"File not found: {audio_file_path}")

In [8]:
res = pipe(audio_file_path)
text = res.get("text")
if text == None:
    logger.warning("No text extracted")
else:
    pprint(text)

" Stress doesn't come, people get stress wrong all the time in my opinion. Stress doesn't come from hard work, for example. You can be working incredibly hard and loving it and likewise you can be out of work and incredibly stressed over that. And likewise if you kind of use that as an analogy for what I was just talking about, if you're out of work but you're going through a disciplined approach of a series of job"

In [14]:
openai_key = None
if 'google.colab' in sys.modules:
    from google.colab import userdata
    openai_key = userdata.get('OPENAI_API_KEY')
else:
    openai_key = os.environ['OPENAI_API_KEY']


In [15]:
prompt = ChatPromptTemplate.from_template(
    """Please provide a summary of the specified {context}.
    The response should be presented as a list of key points,
    formatted in HTML with appropriate markup for clarity and organization.
    """
)
model = ChatOpenAI(openai_api_key=openai_key)
output_parser = StrOutputParser()

In [16]:
chain = prompt | model | output_parser

In [17]:
res = chain.invoke({"context": text})

In [18]:
from IPython.display import display, HTML

display(HTML(res))